# Stokes 2D

Staggered-grid finite-difference implementation of the _Data-Assimilation Example from Glaciology_ given in [1].

In [1]:
import numpy as np
from scipy import optimize, signal
import matplotlib.pyplot as plt

In [2]:
# Parameters
alpha = 0.1/180*np.pi # inclination of the plane
height = 1.0e3        # height of the ice sheet (m)
length = 2.0e4        # length of the domain (m)
nx = 32               # number of grid points in x-direction
ny = 16               # number of grid points in y-direction
rho = 910.0           # density of ice, kg m^-3
mu0 = 1.0             # initial viscosity
g = 9.81              # gravitational acceleration (m s^-2)
A_glen = 1.0e-16      # Glen flow parameter (Pa^-n a^-1)
n_glen = 3            # Glen exponent
N = 30                # number of coefficients in the trigonometric expansion of beta^2

In [3]:
# Grid setup
hx = length/(nx-1)               # grid spacing in x-direction
hy = height/(ny-1)               # grid spacing in y-direction
x = np.linspace(0,length,nx)
y = np.linspace(0,height,ny)
X, Y = np.meshgrid(x,y)
Ux = (X[1:]+X[:-1])/2
Uy = (Y[1:]+Y[:-1])/2
Vx = (X[:,1:]+X[:,:-1])/2
Vy = (Y[:,1:]+Y[:,:-1])/2
Px = (X[1:,1:]+X[:-1,:-1])/2
Py = (Y[1:,1:]+Y[:-1,:-1])/2

In [4]:
# Numbering scheme
scheme = np.zeros((2*ny-1, 2*nx-1), dtype="int")
for i in range(0,2*ny-1):
    for j in range(0,2*nx-1):
        if i%2!=0 and j%2==0: scheme[i,j] = 1 # u
        if i%2==0 and j%2!=0: scheme[i,j] = 2 # v
        if i%2!=0 and j%2!=0: scheme[i,j] = 3 # p
            
# Reshaped vector of indices
n_equations = (2*nx-1)*(2*ny-1)
indices = np.arange(0,n_equations).reshape(2*ny-1, 2*nx-1, order="C")

# Indices of u, v, and p
u_indices = indices[scheme==1]
v_indices = indices[scheme==2]
p_indices = indices[scheme==3]
mu_indices = indices[(scheme==0) | (scheme==3)]

# Indices of the entries of Ux and Uy, Vx and Vy, Px and Py, respectively
U_indices = u_indices.reshape(ny-1, nx)
V_indices = v_indices.reshape(ny, nx-1)
P_indices = p_indices.reshape(ny-1, nx-1)

In [5]:
# Initialize the viscosity mu
mu_glen = mu0*np.ones((2*ny-1, 2*nx-1))
np.put(mu_glen, indices[(scheme==1) | (scheme==2)], 1)
MU_glen = mu_glen.reshape(2*ny-1, 2*nx-1)

# Compute mu at different grid points (1st variant)
# MU_tmp = np.zeros((2*ny+1, 2*nx+1))
# MU_tmp[1:-1,1:-1] = MU_glen
# MU_tmp[:,0] = MU_tmp[:,2]; MU_tmp[:,-1] = MU_tmp[:,-3]
# MU_tmp[0,:] = MU_tmp[2,:]; MU_tmp[-1,:] = MU_tmp[-3,:]
# kernel = np.array([[0,1,0],[1,0,1],[0,1,0]])
# MU_tmp_averages = signal.convolve2d(MU_tmp, kernel, boundary='fill', mode='same')/kernel.sum()
# MU_averages = MU_tmp_averages[1:-1,1:-1]

# Compute mu at different grid points (2nd variant)
kernel = np.array([[0,1,0],[1,0,1],[0,1,0]])
MU_averages = signal.convolve2d(MU_glen, kernel, boundary='fill', mode='same')/kernel.sum()
MU_averages[0,0] *= 2; MU_averages[0,2*nx-2] *= 2                   # fix top left and right entries
MU_averages[1:2*ny-2,0] *= 4/3; MU_averages[1:2*ny-2,2*nx-2] *= 4/3 # fix first and last row
MU_averages[0,1:2*nx-2] *= 4/3; MU_averages[2*ny-2,1:2*nx-2] *= 4/3 # fix first and last col
MU_averages[2*ny-2,0] *= 2; MU_averages[2*ny-2,2*nx-2] *= 2         # fix bottom left and right entries

# Extract the relevant values
mu_averages_flat = MU_averages.flatten()
mu_averages = mu_averages_flat[mu_indices]

In [6]:
# Initialize the vector of unknowns
x_values = np.zeros((n_equations))

In [7]:
# Extract values of U, V, and P from x_values
u_values = np.take(x_values, u_indices)
U_values = u_values.reshape((ny-1), nx)
v_values = np.take(x_values, v_indices)
V_values = v_values.reshape(ny, nx-1)
p_values = np.take(x_values, p_indices)
P_values = p_values.reshape(ny-1, nx-1)

In [8]:
# How to obtain the value of u with a specific index
index = 217
value = u_values[np.where(u_indices==index)[0][0]]

## References

[1] Granzow, G. (2014). A tutorial on adjoint methods and their use for data assimilation in glaciology. Journal of Glaciology, 60(221), 440-446. doi:10.3189/2014JoG13J205

[2] Becker, T. W. and B. J. P. Kaus (2013). Numerical Modeling of Earth Systems. An introduction to computational methods with focus on solid Earth applications of continuum mechanics. Lecture notes, available online at http://www-udc.ig.utexas.edu/external/becker/Geodynamics557.pdf, accessed 01/2018.